<img src="https://github.com/rjpost20/Anomalous-Bank-Transactions-Detection-Project/blob/main/data/AdobeStock_319163865.jpeg?raw=true">
Image by <a href="https://stock.adobe.com/contributor/200768506/andsus?load_type=author&prev_url=detail" >AndSus</a> on Adobe Stock

# Phase 5 Project: *Detecting Anomalous Financial Transactions*

## Notebook 4: Modeling Part 2, Analysis and Results

### By Ryan Posternak

Flatiron School, Full-Time Live NYC<br>
Project Presentation Date: August 25th, 2022<br>
Instructor: Joseph Mata

<br>

# Imports and Reading in Data

### Google colab compatibility downloads

In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz 
!tar xf spark-3.3.0-bin-hadoop3.tgz
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"
!pip install pyspark==3.3.0
!pip install -q findspark
import findspark
findspark.init()

In [ ]:
# Connect to Google drive
from google.colab import drive, files
drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/Colab Notebooks/'

In [ ]:
import numpy as np

from pyspark.sql import SparkSession
import pyspark.sql.functions as F

from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, \
MultilayerPerceptronClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, TrainValidationSplitModel

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

import matplotlib.pyplot as plt
from IPython.display import HTML, display
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
# Import helper functions
helper_functions = files.upload()
from helper_functions import set_weight_col, spark_resample, grid_search, score_model, plot_confusion_matrix

In [ ]:
# Check Colab RAM info
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
# Set text to wrap in Google colab notebook
def set_css():
    display(HTML("""
    <style>
      pre {
          white-space: pre-wrap;
      }
    </style>
    """))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
# Initialize Spark Session
spark = SparkSession.builder\
        .master("local[*]")\
        .appName("Colab")\
        .config("spark.ui.port", "4050")\
        .config("spark.driver.memory", "15g")\
        .getOrCreate()

spark

## Read in Data

In [ ]:
# Read in weighted_df and resampled_df (training data) and test_df_full (testing data) data csv files as Spark DataFrames
train_df_full = spark.read.csv(drive_path + 'train_df_full.csv', header=True, inferSchema=True)
test_df_full = spark.read.csv(drive_path + 'test_df_full.csv', header=True, inferSchema=True)

<br>

## Add `Weight` column to dataframe

As we've seen, the training dataset is extremely imbalanced in regards to target class distribution. In order to improve modeling performance, we'll add a new column `Weight` to `train_df_full` specifying the weights to use, which we pass in to PySpark models in the `weightCol` parameter. We'll create the new `Weight` column using the `set_weight_col` function in `helper_functions`.

Our initial `Weight` column will specify equal weights across all observations, which is the default in PySpark.

In [ ]:
print(set_weight_col.__doc__)

In [ ]:
train_df_full = set_weight_col(train_df_full, label_col='Label', pos_class_weight=1.0, neg_class_weight=1.0)

In [ ]:
# Preview Weight column
cols_to_show = ['MessageId', 'Label', 'Weight']
train_df_full.select(cols_to_show).where(train_df_full.Label == 0).show(1, truncate=False, vertical=True)
train_df_full.select(cols_to_show).where(train_df_full.Label == 1).show(1, truncate=False, vertical=True)

In [ ]:
# Print shape of dataframes
print(f"train_df_full:  {train_df_full.count():,} Rows, {len(train_df_full.columns)} Columns")
print(f"test_df_full:  {test_df_full.count():,} Rows, {len(test_df_full.columns)} Columns")

In [ ]:
# Print schema of training dataframe
print('train_df_full:')
train_df_full.printSchema()

In [ ]:
# Print schema of test dataframe
print('test_df_full:')
test_df_full.printSchema()

In [ ]:
# Drop 'MessageId' individual transaction identifier column - will not be used in modeling
train_df_full = train_df_full.drop('MessageId')
test_df_full = test_df_full.drop('MessageId')

# Rename target variable 'Label' column to more descriptive 'Anomalous'
train_df_full = train_df_full.withColumnRenamed('Label', 'Anomalous')
test_df_full = test_df_full.withColumnRenamed('Label', 'Anomalous')

In [ ]:
# Display first row of train_df_full dataframe
train_df_full.show(n=1, truncate=False, vertical=True)

In [ ]:
# Display first row of test_df_full dataframe
test_df_full.show(n=1, truncate=False, vertical=True)

In [ ]:
# Display value counts for 'Anomalous' column (classification target)
class_counts = train_df_full.groupBy('Anomalous').count().withColumn('percent', F.col('count')/train_df_full.count())

class_counts.show(truncate=False)

<br>

# Create `Pipeline` to Preprocess and Model Data

### Index string columns with `StringIndexer`

In [ ]:
stages = []

categoricalCols = [item[0] for item in train_df_full.dtypes if item[1].startswith('string')]

indexers = []
for col in categoricalCols:
    indexer = StringIndexer(inputCol=col, outputCol=col + '_index', handleInvalid='keep')
    indexers.append(indexer)
    
indexed_features = []
for si in indexers:
    indexed_features.append(si.getOutputCol())
    
print(f"Indexed nominal categorical features: {len(indexed_features)}")
print(indexed_features)

### Create a `OneHotEncoder` to encode the indexed string features

In [ ]:
encoder = OneHotEncoder(inputCols=indexed_features, 
                        outputCols=[col + '_ohe' for col in indexed_features], 
                        dropLast=True)

print(f"One hot encoded nominal categorical features: {len(encoder.getOutputCols())}")
print(encoder.getOutputCols())

### Compile numeric features, not including target or class weight columns

In [ ]:
numeric_features = []
for column, dtype in train_df_full.dtypes:
    if column != 'Anomalous' and column != 'Weight' and dtype != 'string':
        numeric_features.append(column)

# Confirm equal column counts
assert len(train_df_full.drop('Anomalous', 'Weight').columns) == len(indexed_features) + len(numeric_features)
print(f"Numeric features: {len(numeric_features)}\n{numeric_features}")

In [ ]:
# Print names of final features going into the model
features = encoder.getOutputCols() + numeric_features
print(f"Final features: \n{features}")

### Create a `VectorAssembler` to combine all features

In [ ]:
assembler = VectorAssembler(inputCols=features, outputCol='vectorized_features')

# Assemble a list of stages that includes the vector assembler and standard scaler
scaler = StandardScaler(inputCol='vectorized_features', outputCol='scaled_features')

stages = indexers + [encoder, assembler, scaler]
print("Stages:", stages)

### Preview modeling pipeline

In [ ]:
pipeline = Pipeline(stages=stages)

pipeline_model = pipeline.fit(train_df_full)

pipeline_df = pipeline_model.transform(train_df_full)

In [ ]:
# Display first row of train_df_full after running through pipeline
pipeline_df.show(1, vertical=True, truncate=False)

In [ ]:
pipeline_test = pipeline.fit(test_df_full)

pipeline_df_test = pipeline_model.transform(test_df_full)

pipeline.fit(test_df_full).transform(test_df_full).head()['scaled_features'].size

In [ ]:
# Display first row of test_df_full after running through pipeline
pipeline_df_test.show(1, vertical=True, truncate=False)

<br>

# **Baseline Model**: Decision Tree Classifier with Tuned Max Depth

For our baseline model on the full training dataset, we'll use the best classifier from notebook-3, which was a `DecisionTreeClassifier`.

In [ ]:
dt_1 = DecisionTreeClassifier(
    featuresCol='scaled_features',
    labelCol='Anomalous',
    weightCol='Weight')

dt_1_stages = stages + [dt_1]

# Specify parameter grid
dt_1_grid = ParamGridBuilder()\
            .addGrid(dt_1.maxDepth, [2, 3, 4, 5, 6])\
            .build()

In [ ]:
%%time
# Run grid search using grid_search function
if not os.path.isdir(drive_path + 'dt_1_model_full'):
    dt_1_model = grid_search(stages_with_classifier=dt_1_stages, 
                             train_df=train_df_full, 
                             model_grid=dt_1_grid, 
                             parallelism=5)

In [ ]:
# Save model, or upload if already saved
if not os.path.isdir(drive_path + 'dt_1_model_full'):
    dt_1_model.save(drive_path + 'dt_1_model_full')
else:
    dt_1_model = TrainValidationSplitModel.load(drive_path + 'dt_1_model_full')
    print(dt_1_grid[np.argmax(dt_1_model.validationMetrics)])

In [ ]:
# Print model scores
score_model(dt_1_model, train_df_full, test_df_full)

In [ ]:
# Plot model confusion matrix
plot_confusion_matrix(dt_1_model, test_df_full)